In [ ]:
import re
import numpy as np
import pandas as pd
import sqlalchemy
import os
import logging

from sqlalchemy import create_engine
from dotenv import load_dotenv

from hydra import compose, initialize

from hdb_resale import sql, utils

pd.set_option('display.max_columns', None)

In [ ]:
# Retrieve environment variables
load_dotenv()

BING_MAP_API_KEY = os.environ.get("BING_MAP_API_KEY")
POSTGRESQL_DASH_USER = os.environ.get("POSTGRESQL_DASH_USER")
POSTGRESQL_DASH_PASSWORD = os.environ.get("POSTGRESQL_DASH_PASSWORD")
POSTGRESQL_DASH_DATABASE = os.environ.get("POSTGRESQL_DASH_DATABASE")
POSTGRESQL_HOST = os.environ.get("POSTGRESQL_HOST")
POSTGRESQL_PORT = os.environ.get("POSTGRESQL_PORT")

# Setup logger
logger = logging.getLogger(__name__)

# Create the connection string
connection_string = f"postgresql+psycopg2://{POSTGRESQL_DASH_USER}:{POSTGRESQL_DASH_PASSWORD}@{POSTGRESQL_HOST}:{POSTGRESQL_PORT}/{POSTGRESQL_DASH_DATABASE}"

# Create a SQLAlchemy engine
engine = create_engine(connection_string)

In [ ]:
with initialize(version_base=None, config_path="../../airflow_prd/dags/conf"):
    cfg = compose(config_name="hdb_resale_config")

In [ ]:
engine, metadata = sql.setup_database(
    postgresql_dash_user=POSTGRESQL_DASH_USER, 
    postgresql_dash_password=POSTGRESQL_DASH_PASSWORD, 
    postgresql_dash_database=POSTGRESQL_DASH_DATABASE,
    postgresql_host=POSTGRESQL_HOST,
    postgresql_port=POSTGRESQL_PORT
)

# One off data table migration

In [ ]:
# Migrate town data

town_query = sqlalchemy.text("""
    SELECT 
        town, census_mapping, total_population, 
        population_below24, population_2559, population_above60, 
        num_hdb, num_condo, num_landed, num_others, 
        income_below4k, income_4k8k, income_8k12k, income_above12k, 
        household_wodisability, household_withdisability, 
        pct_population_below24, pct_population_2559, pct_population_above60, 
        pct_hdb, pct_condo, pct_landed, 
        pct_income_below4k, pct_income_4k8k, pct_income_8k12k, pct_income_above12k,
        pct_household_wodisability, pct_household_withdisability, 
        lat, lng 
    FROM hdb_resale.town
""")
town_data = pd.read_sql(town_query, con=engine)

town_data["source_name"] = "singstat_census_2020"

In [ ]:
# Create row hash identifier using key columns
town_data["_row_hash_id"] = town_data[cfg.database.town_data.hash_key_columns].apply(utils.create_hash, axis=1)

# Remove data based on primary key
# Needs to be done before data insertion to prevent database duplicated errors
# No data will be removed if the primary key does not exist
sql.delete_data_primary_key(
    engine=engine,
    metadata=metadata,
    schema_table_name=cfg.database.town_data.schema_table_name,
    primary_key=town_data["_row_hash_id"].to_list(),
)

# Insert data into database
with engine.connect() as con:
    town_data.to_sql(
        name=cfg.database.town_data.table_name, schema=cfg.database.town_data.schema_name, con=con, if_exists="append", index=False, chunksize=10000
    )

logger.info(f"Loaded {town_data.shape[0]} rows of data into table {cfg.database.town_data.schema_table_name}.")

In [ ]:
# Migrate address data

address_query = sqlalchemy.text("""
    SELECT 
        block, street_name, town, 
        country, full_address, 
        lat, lng 
    FROM hdb_resale.geolocation
""")
address_data = pd.read_sql(address_query, con=engine)

address_data["source_name"] = "bing_map_api"

In [ ]:
# Create row hash identifier using key columns
address_data["_row_hash_id"] = address_data[cfg.database.address_data.hash_key_columns].apply(utils.create_hash, axis=1)

# Remove data based on primary key
# Needs to be done before data insertion to prevent database duplicated errors
# No data will be removed if the primary key does not exist
sql.delete_data_primary_key(
    engine=engine,
    metadata=metadata,
    schema_table_name=cfg.database.address_data.schema_table_name,
    primary_key=address_data["_row_hash_id"].to_list(),
)

# Insert data into database
with engine.connect() as con:
    address_data.to_sql(
        name=cfg.database.address_data.table_name, schema=cfg.database.address_data.schema_name, con=con, if_exists="append", index=False, chunksize=10000
    )

logger.info(f"Loaded {address_data.shape[0]} rows of data into table {cfg.database.address_data.schema_table_name}.")

# Process town data from file

In [ ]:
logger.info("Reprocessing town data......")

# Read all data from a single file
# NOTE it can currently only read from a single file
data = pd.read_csv(cfg.town_data_path)

# Calculate proportions for all numbers

total_population = data["population_below24"] + data["population_2559"] + data["population_above60"]
data["pct_population_below24"] = data["population_below24"] / total_population
data["pct_population_2559"] = data["population_2559"] / total_population
data["pct_population_above60"] = data["population_above60"] / total_population

total_property = data["num_hdb"] + data["num_condo"] + data["num_landed"]
data["pct_hdb"] = data["num_hdb"] / total_property
data["pct_condo"] = data["num_condo"] / total_property
data["pct_landed"] = data["num_landed"] / total_property

total_income = data["income_below4k"] + data["income_4k8k"] + data["income_8k12k"] + data["income_above12k"]
data["pct_income_below4k"] = data["income_below4k"] / total_income
data["pct_income_4k8k"] = data["income_4k8k"] / total_income
data["pct_income_8k12k"] = data["income_8k12k"] / total_income
data["pct_income_above12k"] = data["income_above12k"] / total_income

total_disability = data["household_wodisability"] + data["household_withdisability"]
data["pct_household_wodisability"] = data["household_wodisability"] / total_disability
data["pct_household_withdisability"] = data["household_withdisability"] / total_disability

# Read geocoded data
logger.info("Loading geocoded address data......")
add_query = sqlalchemy.text("SELECT * FROM hdb_resale.geolocation")
add_data = pd.read_sql(add_query, con=engine)

# Get median lat & lng per town
# Needed for predictions
median_add = add_data.groupby(by=["town"], as_index=False).agg({"lat": "median", "lng": "median"})

data = data.merge(
    median_add[["town", "lat", "lng"]],
    how="left",
    on=["town"],
)

# # Write data to database
# schema_name = "hdb_resale"
# table_name = "town"
# delete_table(schema=schema_name, table=table_name, con=engine)
# data.to_sql(name=table_name, con=engine, schema=schema_name, index=False, if_exists="append")
